In [8]:
!pip install pydicom
!pip install -U pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg
!pip install pylibjpeg-libjpeg

In [9]:
import numpy as np
import pandas as pd
import os
import sys
from glob import glob
import cv2
import re
import gc
import pydicom as dicom
import nibabel as nib
from tqdm import tqdm

In [10]:
save_list = [
    '1.2.826.0.1.3680043.1363',
    '1.2.826.0.1.3680043.4202',
    '1.2.826.0.1.3680043.26979',
    '1.2.826.0.1.3680043.30067'
]

In [11]:
def load_dicom(inpath: str, size: int=512):
    paths = sorted(inpath, key=lambda p: int(os.path.splitext(os.path.basename(p))[0]))
    
    imgs = []
    for path in paths:
        img = dicom.read_file(path)
        img = img.pixel_array
        img = cv2.resize(img, (size, size), interpolation=cv2.INTER_LINEAR)
        img = img - np.min(img)
        img = img / (np.max(img) + 1e-7)
        img = (img * 255).astype(np.uint8)
        imgs.append(img)
        
    return imgs

In [12]:
def load_nii(path, size=512):
    segment = nib.load(path).get_fdata() # convert to numpy array
    # conversion from axial to sagittal view
    seg = segment[:, ::-1, ::-1].transpose(1, 0, 2)
    shape = seg.shape
    seg = cv2.resize(seg, (size, size), interpolation=cv2.INTER_LINEAR)
    seg = seg.astype(np.uint16) * 255
    mask = np.asarray(seg)

    # image orientation: (width, height, channel)
    return mask.transpose(1, 2, 0)

In [13]:
try:
    os.mkdir('train_images')
    os.mkdir('segmentations')
except: pass

train_df = pd.read_csv('/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train.csv')
for idx in range(len(train_df)):
    uid = train_df.loc[idx]['StudyInstanceUID']
    if uid in save_list:
        image_path = glob(f'/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/{uid}/*.dcm')
        mask_path = f'/kaggle/input/rsna-2022-cervical-spine-fracture-detection/segmentations/{uid}.nii'
        
        image = load_dicom(image_path)
        mask = load_nii(mask_path)
        
        for idx in range(len(image)):
            image_save_path = f'/kaggle/working/train_images/{uid}_{idx}.npz'
            np.savez_compressed(image_save_path, image[idx])
            
            mask_save_path = f'/kaggle/working/segmentations/{uid}_{idx}.npz'
            np.savez_compressed(mask_save_path, mask[idx])

In [14]:
import shutil

# compress 'prep_seg' dir as a zip file
output_filename = 'train_images'
dir_name = 'train_images'
shutil.make_archive(output_filename, 'zip', dir_name)
path = '/kaggle/working/train_images'


# remove all files in prep_seg directory
for file_name in os.listdir(path):
    file = path + '/' + file_name
    if os.path.isfile(file):
        os.remove(file)
# delete uncompressed directory
os.rmdir('train_images')


# compress 'prep_seg' dir as a zip file
output_filename = 'segmentations'
dir_name = 'segmentations'
shutil.make_archive(output_filename, 'zip', dir_name)
path = '/kaggle/working/segmentations'

# remove all files in prep_seg directory
for file_name in os.listdir(path):
    file = path + '/' + file_name
    if os.path.isfile(file):
        os.remove(file)
# delete uncompressed directory
os.rmdir('segmentations')